In [ ]:
import pandas as pd

# Carregar  dados da camada bronze
df = pd.read_parquet('data/bronze/dados_brutos.parquet')
print(f"Dados originais {df.shape}")
df_clean = df.copy()

# Transformações

## 1. Valores Faltantes

### CustomerID




*   CustomerID	| 135.080 Rows NaN

---


Checando a possibilidade de tratar CustomerID NaN de acordo com InvoiceNo (Nº da fatura) iguais e com CustomerID preenchido

In [ ]:
# Contar quantos InvoiceNo possuem pelo menos um CustomerID nulo
invoice_null = df.groupby('InvoiceNo')['CustomerID'].apply(lambda x: x.isnull().any())
invoice_notnull = df.groupby('InvoiceNo')['CustomerID'].apply(lambda x: x.notnull().any())

# Quantos têm mistura (nulo e não nulo)?
mixed_invoice = (invoice_null & invoice_notnull)

print("Total de InvoiceNo mistos:", mixed_invoice.sum())
print("Total de InvoiceNo com CustomerID nulo:", invoice_null.sum())
print("Total de InvoiceNo com CustomerID não nulo:", invoice_notnull.sum())
print(f"Total de linhas com CustomerID nulo: {df['CustomerID'].isnull().sum()}")

 - Total de InvoiceNo mistos: 0


Significa que nao é possivel recuperar CustomerID pelo InvoiceNo

### Description

*   Description | 1454 Rows NaN

 Muitos Description possuem erros de preenchimento também

---



Cada produto possui o seu StockCode. Muitos Description (nome do produto) estão NaN ou preenchidos de forma errada, porém possuem StockCode iguais

In [ ]:
# Contar quantos StockCode possuem pelo menos um Description nulo
StockCode_null = df.groupby('StockCode')['Description'].apply(lambda x: x.isnull().any())
StockCode_notnull = df.groupby('StockCode')['Description'].apply(lambda x: x.notnull().any())

# Quantos têm mistura (nulo e não nulo)?
mixed_StockCode = (StockCode_null & StockCode_notnull)

print("Total de StockCode mistos:", mixed_StockCode.sum())
print("Total de StockCode com Description nulo:", StockCode_null.sum())
print("Total de StockCode com Description não nulo:", StockCode_notnull.sum())
print(f"Total de linhas com Description nulo: {df['Description'].isnull().sum()}")
print("==========================================")

#Exemplo:
display(df[df["StockCode"] == "35965"])
print("\n")
print(f"Quantidade de Description NaN do StockCode 35965: {df[df['StockCode'] == '35965']['Description'].isnull().sum()}")

In [ ]:
# Criar mapeamento de StockCode  Description válida
mapa_descricoes = df_clean.dropna(subset=['Description']).groupby('StockCode')['Description'].first().to_dict()

# Preencher Description de acordo com o primeiro valor
df_clean['Description'] = df_clean.apply(
    lambda row: mapa_descricoes.get(row['StockCode'], row['Description']),
    axis=1
)

print(f"Descriptions recuperados: {df['Description'].isnull().sum() - df_clean['Description'].isnull().sum()}")
print(f"Descriptions não recuperados: {df_clean['Description'].isnull().sum()}")
print(f"Quantidade de Descriptions erradas corrigidas: {df['Description'].nunique() - df_clean['Description'].nunique()}")

##  2. Duplicatas

 Duplicatas são aceitáveis no modelo de negócio deste DataFrame. Cada linha representa um item em uma fatura, portanto, o mesmo `InvoiceNo` (número da fatura) pode aparecer várias vezes se uma fatura contiver múltiplos produtos. A venda em si é representada unicamente pelo `InvoiceNo`.

In [ ]:
print(f"Quantidade de linhas duplicadas: {df_clean.duplicated().sum()}")
display(df_clean[df_clean.duplicated()])

## 3. Valores negativos, Cancelamentos e tarifas

In [ ]:
# Linhas de tarifas
stockcode_fees = ['C2', 'DOT', 'POST','AMAZONFEE']

# Filtrar DataFrame para mostrar Inconsistências:
#(InvoiceNo que começam com 'C','A)'| Quantity <= 0 | UnitPrice <= 0 e StockCode de tarifas
df_inconsistencias = df_clean[
   (df_clean['InvoiceNo'].astype(str).str.startswith('C')) |
    (df_clean['InvoiceNo'].astype(str).str.startswith('A')) |
     (df_clean['Quantity'] <= 0) | (df_clean['UnitPrice'] <= 0) | (df_clean['StockCode'].isin(stockcode_fees))
]

display(df_inconsistencias)
print("Dados serão divididos na camada gold")

## 4. Outliers

In [ ]:
df_clean.describe()

In [ ]:
# identificando Outliers
display(df_clean[df_clean['Quantity'] > 5000])

print("\n")
print("-------------------------------------------------------")
print("Verificando padrão de compra do maior Outlier/Cliente")
print("-------------------------------------------------------")

# Trocar ID para visualizar todos
display(df_clean[df_clean['CustomerID'] == 16446])

print("Foram feitos cancelamenos das compras Outliers")

In [ ]:
# Testar outliers usando IQR para a coluna 'Quantity'
Q1 = df_clean['Quantity'].quantile(0.25)
Q3 = df_clean['Quantity'].quantile(0.75)
IQR = Q3 - Q1

outliers_iqr = df_clean[(df_clean['Quantity'] < (Q1 - 1.5 * IQR)) | (df_clean['Quantity'] > (Q3 + 1.5 * IQR))]
print(f"Quantidade de outliers detectados pelo IQR em Quantity: {outliers_iqr.shape[0]}")
display(outliers_iqr)

Resultado de 58.169 outliers não fazem sentido real.
A maior parte deles são possiveis compras grandes legítimas.

variável Quantity tem estas características:

- Distribuição extremamente assimétrica (muitos valores baixos, poucos valores muito altos);
 
- Existem cancelamentos e devoluções com valores negativos;

- Existem compras corporativas (de 500, 1000, 2000 unidades) perfeitamente legítimas;

- E há linhas com erros humanos ou transações de teste, mas elas são exceções.

## 5. Carregando dados na camada Silver


In [ ]:
df_clean.to_parquet("data/silver/dados_limpos.parquet",index=False)
print("Dados salvos na camada Silver")